In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
from selenium.webdriver import Chrome

In [2]:
from selenium.webdriver.chrome.service import Service
service = Service(ChromeDriverManager().install())

In [3]:
def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = random.uniform(1, 2)
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

In [4]:
# 검색 키워드 설정: 키워드 내 띄어쓰기는 URL에서 '+'로 표시되기 때문에 이에 맞게 변환
SEARCH_KEYWORD = '침착맨'.replace(' ', '+')

In [5]:
driver = Chrome(ChromeDriverManager().install())
# 스크래핑 할 URL 세팅
URL = "https://www.youtube.com/results?search_query=" + SEARCH_KEYWORD
#URL = 'https://www.youtube.com/@calmdownman_official/videos'
# 크롬 드라이버를 통해 지정한 URL의 웹 페이지 오픈
driver.get(URL)
# 웹 페이지 로딩 대기
time.sleep(3)
# 무한 스크롤 함수 실행
scroll()

스크롤 완료


In [6]:
# 페이지 소스 추출
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

In [7]:
# 콘텐츠 모든 정보
content_total = soup_source.find_all(class_ = 'yt-simple-endpoint style-scope ytd-video-renderer')
# 콘텐츠 제목만 추출
content_total_title = list(map(lambda data: data.get_text().replace("\n", ""), content_total))
# 콘텐츠 링크만 추출
content_total_link = list(map(lambda data: "https://youtube.com" + data["href"], content_total))

#--------조회수 & 업로드 날짜 추출(Updated at 2022-10-11)--------#
content_record_src = soup_source.find_all(class_ = 'style-scope ytd-video-meta-block')
content_view_cnt = [content_record_src[i].get_text().replace('조회수 ', '') for i in range(5, len(content_record_src), 10)]
content_upload_date = [content_record_src[i].get_text() for i in range(6, len(content_record_src), 10)]
#---------------------------------------------------------#

# 딕셔너리 포맷팅
content_total_dict = {'title'       : content_total_title, 
                      'link'        : content_total_link
                     }

In [8]:
df = pd.DataFrame(content_total_dict)
df

,title,link
0,모든 분리수거의 신,https://youtube.com/watch?v=PhWyC9sPF0M
1,(잔인주의) 선량한 사람은 너굴맨이 처리했으니 안심하라구,https://youtube.com/watch?v=8hSds3puPQY
2,계속되는 남도형 성우와의 토크,https://youtube.com/watch?v=mdsaZz5WVHU
3,"2023년 02월 10일 | 카더가든 · 비비 초대석, 러스티 레이크 루츠, 학창시...",https://youtube.com/watch?v=cYOqd3GINa4
4,침&펄 | 아이돌 인물 퀴즈 & 수도 퀴즈,https://youtube.com/watch?v=S9syINFA5Zo
...,...,...
97,침착맨이 알려주는 고사성어,https://youtube.com/watch?v=Vrl78SZ724c
98,'블랙홀의 과학' 특강 | 제1부 강의의 지평선 너머로,https://youtube.com/watch?v=BYW59WPeBss
99,듣기 싫은 꼰대어 월드컵,https://youtube.com/watch?v=gcRzKVztwK4
100,침착맨 3월 10일부터 휴방 발표,https://youtube.com/watch?v=Yg1zmWl1wO0


In [9]:
df.to_csv("침착맨")

In [10]:
driver.close()